<a href="https://colab.research.google.com/github/nattaponkum/DeepSpeed-MII/blob/main/chatbot_whatsapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is for demonstration purpose only. It does not claim to be especially elegant or efficient.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 20 09:30:08 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Reading and processing the data

In [2]:
# You need to re-boot the instance after installing
!pip install whatstk
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 61.0 MB/s eta 0:00:00
  Created wheel for whatstk: filename=whatstk-0.6.2-py3-none-any.whl size=60396 sha256=c1a2359ab7c919377ed1bbdd21b95dc7eadfe5683b23802f965f5b752ebdac1d
  Stored in directory: /root/.cache/pip/wheels/62/1e/f2/69386e57a388ed2a9f8e19db1cc95ccd735dd6a521dd686b00
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234912 sha256=4336c93d8cfa84b7cb5fcd2492d0899b54c09b949f609ddc2f13c90c7642590e
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built whatstk emoji
  Attempting uninstall: plotly
    Found exis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00


In [1]:
import pandas as pd
from whatstk import WhatsAppChat
from datetime import timedelta

In [2]:
filepath = "_chat.txt" # you need to copy your export
chat = WhatsAppChat.from_source(filepath=filepath)
df = chat.df

FileNotFoundError: ignored

In [ ]:
# Filter GIFs, stickers and images; works only for German
df = df[df["message"]!="\u200eGIF weggelassen"]
df = df[df["message"]!="\u200eSticker weggelassen\n\u200e"]
df = df[df["message"]!="\u200eBild weggelassen"]

In [ ]:
# Calculate time passed since previous message
df["date_previous"] = df["date"].shift(periods=1)
df["time_delta"] = (df["date"]-df["date_previous"]).dt.total_seconds()

# Concat message and author
df["chat_message"] = df["username"] + ": " + df["message"]

# Remove first line, its just a WhatApp test line
df = df[1:]

In [ ]:
# Convert messages into conversations (a conversation has multiple messages); ugly programming, but works for small data
# Step 1: Concat each message with the previous conversation
query = []
answer = []
conversation = ""
session_ix = 0
sessions_ixs = []

for ix, row in df.iterrows():
  if row["time_delta"]<3600: # This defines on how close messages should be to be in the same conversation
    session_ix = session_ix + 1
    sessions_ixs.append(session_ix)
    if conversation == "":
      conversation = row["chat_message"]
      query.append(conversation)
      answer.append("")
    else:
      conversation = conversation + "| " + row["chat_message"]
      query.append(conversation)
      answer.append(row["chat_message"])
  else:
    session_ix=0
    conversation = ""


df_model = pd.DataFrame({"query": query[:-1], "answer": answer[1:], "session_ix": sessions_ixs[:-1]})

# Step 2: Filter only for the last message of the conversation (therefore for the full conversation.
df_model["model_helper_idx"] = df_model["session_ix"] - df_model["session_ix"].shift(-1)
df_model = df_model[df_model["model_helper_idx"]>-1]

# This way is a bit clumsy, but I did some test with the intermediate conversation steps.

# Fine-tune model

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 18.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import Dataset
import transformers


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:

/usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:

Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of the

In [ ]:
# Import base model
model_id = "malteos/bloom-6b4-clp-german"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# Apply PEFT (Adapter, LoRA and others)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3932160 || all params: 3231490048 || trainable%: 0.1216825656768973


In [ ]:
# Create dataset from pandas Dataframe
dataset = Dataset.from_pandas(df_model)
tokenizer.pad_token = tokenizer.eos_token
ds = dataset.map(lambda samples: tokenizer(samples["query"], truncation=True, padding=True, max_length=512), batched=True)

Map:   0%|          | 0/3471 [00:00<?, ? examples/s]

In [ ]:
# Callback to print intermediate model outputs
class GenerationCallback(transformers.TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 5 == 0:
          input_text = "Daniel: Hey, wie gehts?, Lisa:"
          input_ids = self.tokenizer.encode(input_text, return_tensors="pt")
          generated_ids = self.trainer.model.generate(input_ids=input_ids.to('cuda'))
          generated_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
          print("Generated Text:", generated_text)

# Create an instance of your custom callback
callback = GenerationCallback()

In [ ]:
# Train, make sure to adjust hyperparams
trainer = transformers.Trainer(
    model=model,
    train_dataset=ds,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=120,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

callback.trainer = trainer
callback.tokenizer = tokenizer
trainer.add_callback(callback)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# Make predictions

In [ ]:
def answer(input_text, model, tokenizer):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    generated_ids = model.generate(input_ids=input_ids.to('cuda'), max_new_tokens=20, min_new_tokens=10, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    # Process or use the generated text as needed
    return generated_text

In [ ]:
def chat(role, nonrole, model, tokenizer):
    for i in range(1, 100):
        query = input()
        if i==1:
            query = f"{role}: {query}| {nonrole}:"
        if i>1:
            query = response + f"| {role}: {query}| {nonrole}:"
        response = "| ".join(answer(query, model, tokenizer).split("| ")[:i*2])
        print(response)

In [ ]:
chat("Daniel", "Lisa", model, tokenizer)